<div> 
<img src="https://res-5.cloudinary.com/crunchbase-production/image/upload/c_lpad,h_256,w_256,f_auto,q_auto:eco/v1455514364/pim02bzqvgz0hibsra41.png" align="right"><br><br><FONT FACE="times new roman" SIZE=5>
<b></b>
<br>
<i><b>Docente:</b></i> Juan Pablo Ospina.
<br>
<i><b>Asignatura:</b></i> Vida Artificial.
<br>
<i><b>Estudiante:</b></i> Diana Carolina Quintero Bedoya
<br>
<i><b>Estudiante:</b></i> Juan Diego Varon
<br>
<i><b>Tema:</b></i> Laboratorio 6. ALgoritmo Genético

</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import collections
import random
import seaborn as sn
import statistics

In [ ]:
class Poblacion:
  def __init__(self, seleccion, cruce, mutacion, numPoblacion, tamIndividuo, pbb):
    self.poblacion = None
    self.numPoblacion = numPoblacion
    self.tamIndividuo = tamIndividuo
    self.pbb = pbb
    self.fitness = None
    self.seleccion = Seleccion()
    self.cruce = Cruce()
    self.mutacion = Mutacion()
    self.analisisfitness = Analisis()
    self.visualizacion = Visualizacion()

  def crear(self):
    CreaPoblacion = np.random.choice([0, 1], size=(self.numPoblacion, self.tamIndividuo), p=[1-self.pbb, self.pbb])
    self.fitness = np.sum(CreaPoblacion, axis=1)
    self.poblacion = np.column_stack((CreaPoblacion, self.fitness))

In [ ]:
class Seleccion:

  def seleccionRuleta(poblacion, fitness):
    totalFitness = np.sum(fitness)
    probabilidadSeleccion = fitness / totalFitness
    poblacionSeleccionada = np.random.choice(poblacion.shape[0], size=1, p=probabilidadSeleccion)
    return np.array(poblacion[poblacionSeleccionada])

  def seleccionMuestreoUniversalEstocastico(poblacion, fitness):
    totalFitness = np.sum(fitness)
    probabilidadSeleccion = fitness / totalFitness
    nuevap = np.random.choice(poblacion.shape[0], size=2, p=probabilidadSeleccion)
    poblacionSeleccionada = poblacion[nuevap]
    while collections.Counter(poblacionSeleccionada[0]) == collections.Counter(poblacionSeleccionada[1]):
      nuevap = np.random.choice(poblacion.shape[0], size=2, p=probabilidadSeleccion)
      poblacionSeleccionada = poblacion[nuevap]
    return np.array(poblacionSeleccionada)

  def seleccionRankings(poblacion, fitness):
    rangos = np.argsort(np.argsort(-fitness))
    probabilidadSeleccion = (rangos + 1) / np.sum(rangos + 1)
    poblacionSeleccionada = np.random.choice(poblacion.shape[0], size=1, p=probabilidadSeleccion)
    return np.array(poblacion[poblacionSeleccionada])

  def seleccionTorneo(poblacion, fitness):
    n = len(poblacion)
    seleccionados = []
    for i in range(3):
      seleccion = np.random.choice(n, 3, replace=False)
      ganador = seleccion[np.argmax(fitness[seleccion])]
    return np.array(poblacion[ganador])

In [ ]:
class Cruce:

  def crucePunto(padre1, padre2, puntoCruce):
    hijo = np.concatenate((padre1[:puntoCruce], padre2[puntoCruce:]))
    hijo_fitness = np.sum(hijo)
    hijo = np.append(hijo, hijo_fitness)
    return hijo

  def crucePuntoBoble(padre1, padre2, puntoCruce1, puntoCruce2):
    hijo = np.concatenate((padre1[:puntoCruce1], padre2[puntoCruce1:puntoCruce2] ,padre1[puntoCruce2:]))
    hijo_fitness = np.sum(hijo)
    hijo = np.append(hijo, hijo_fitness)
    return hijo

  def cruceUniforme(individuo1, individuo2):
    mascara = np.random.choice([False, True], size=individuo1.shape)
    hijo = np.where(mascara, individuo1, individuo2)
    hijo_fitness = np.sum(hijo)
    hijo = np.append(hijo, hijo_fitness)
    return hijo

In [ ]:
class Mutacion:

  def mutacionFlipBit(individuo, pbb):
    for i in range(len(individuo)):
        if np.random.random() < pbb:
            individuo[i] = 1 - individuo[i]
    individuo_fitness = np.sum(individuo)
    individuo = np.append(individuo, individuo_fitness)
    return individuo
  
  def mutacionIntercambio(individuo, pbb):
    genoma = individuo.copy()
    posiciones = np.random.choice(genoma.shape[0], size=2, replace=False)
    pos_1 = posiciones[0]
    pos_2 = posiciones[1]
    if np.random.uniform() < pbb:
        genoma[pos_1], genoma[pos_2] = genoma[pos_2], genoma[pos_1]
    genoma_fitness = np.sum(genoma)
    genoma = np.append(genoma, genoma_fitness)
    return genoma

  def mutacionScramble(individuo, pbb):
    nuevo_individuo = individuo.copy()
    if np.random.random() < pbb:
        punto_1 = np.random.randint(0, len(individuo))
        punto_2 = np.random.randint(0, len(individuo))
        while punto_2 == punto_1:
            punto_2 = np.random.randint(0, len(individuo))
        if punto_1 > punto_2:
            punto_1, punto_2 = punto_2, punto_1
        segmento = nuevo_individuo[punto_1:punto_2+1]
        np.random.shuffle(segmento)
        nuevo_individuo[punto_1:punto_2+1] = segmento
    nuevo_individuo_fitness = np.sum(nuevo_individuo)
    nuevo_individuo = np.append(nuevo_individuo, nuevo_individuo_fitness)
    return nuevo_individuo

  def mutacion_inversion(individuo, pbb):
    if np.random.rand() < pbb:
        punto1 = np.random.randint(len(individuo))
        punto2 = np.random.randint(len(individuo))
        while punto1 == punto2:
            punto2 = np.random.randint(len(individuo))
        if punto1 > punto2:
            punto1, punto2 = punto2, punto1
        segmento = individuo[punto1:punto2+1]
        individuo[punto1:punto2+1] = segmento[::-1]
    individuo_fitness = np.sum(individuo)
    individuo = np.append(individuo, individuo_fitness)
    return individuo

In [ ]:
class Analisis:
  
  def __init__(self, poblacion):
    self.poblacion = poblacion

  # Media
  def media(self):
    return np.mean(self.poblacion)

  # Varianza  
  def varianza(self):
    return np.var(self.poblacion)

  # Desviación estandar
  def desviacion_estandar(self):
    return np.std(self.poblacion)

  # Máximo
  def maximo(self):
    return np.max(self.poblacion)

  # Mínimo
  def minimo(self):
    return np.min(self.poblacion)
    
  def resumen(self):
    print("Resumen análsis descriptivo básico sobre el Fitness de los individuos:")
    print("Media: {:.2f}".format(self.media()))
    print("Varianza: {:.2f}".format(self.varianza()))
    print("Desviación estándar: {:.2f}".format(self.desviacion_estandar()))
    print("Máximo: {:.2f}".format(self.maximo()))
    print("Mínimo: {:.2f}".format(self.minimo()))

tam_poblacion = 10000000

tam_individuo = 10

probabilidad = 0.5

poblacion_fitness = np.random.choice([0, 1], size=(tam_poblacion, tam_individuo), p=[1-probabilidad, probabilidad])

analisispobla = Analisis(poblacion_fitness)
analisispobla.resumen()

Resumen análsis descriptivo básico sobre el Fitness de los individuos:
Media: 0.50
Varianza: 0.25
Desviación estándar: 0.50
Máximo: 1.00
Mínimo: 0.00


In [71]:
class Visualizacion:
  
  def __init__(self, poblacion, estadistics):
    self.poblacion = poblacion  
    self.estadistics = estadistics
    
  # Fitness Promedio de la Población vs Numero de Generaciones
  def fitness_prom_gen(self):
    plt.plot(np.arange(len(self.estadistics['Promedio']))+1, self.estadisticas['mean_fitness'])
    plt.title('Fitness Promedio de la Población vs Numero de Generaciones')
    plt.xlabel('Número de Generaciones')
    plt.ylabel('Fitness Promedio')
    plt.show()

  # Diagrama de Cajas 
  def diagrama_cajas(self):
    plt.boxplot(self.poblacion.fitness)
    plt.ylabel('Fitness')
    plt.title('Diagrama de cajas del fitness de la población')
    plt.show()

  # Histograma de Fitness 
  def histograma(self):
    plt.hist(self.poblacion.fitness, bins=50)
    plt.xlabel('Fitness')
    plt.ylabel('Frecuencia')
    plt.title('Histograma del fitness de la población')
    plt.show()
  
  # Fitness del mejor y peor individuo de la población
  def mejor_peor_fitness(self):
    plt.plot(np.arange(len(self.estadistics['Mejor']))+1, self.estadisticas['Mejor'], label='Mejor')
    plt.plot(np.arange(len(self.estadistics['Peor']))+1, self.estadisticas['Peor'], label='Peor')
    plt.title('Fitness del Mejor y Peor Individuo de la Población vs Numero de Generaciones')
    plt.xlabel('Generación')
    plt.ylabel('Fitness')
    plt.legend()
    plt.show()

numPoblacion = 100
tamIndividuo = 5
poblacion=1000
pbb = 0.5

#poblacion = Poblacion(numPoblacion, tamIndividuo)
#poblacion.crear()

estadistics = {
    'Promedio': [np.mean(self.poblacion.fitness)],
    'Mejor': [np.max(poblacion.fitness)],
    'Peor': [np.min(poblacion.fitness)]
}

# Create an instance of the Visualizacion class
visualizacion = Visualizacion(poblacion, estadistics)

# Create a line plot of the average fitness over generations
visualizacion.fitness_prom_gen()

NameError: ignored

In [ ]:
def crear(numPoblacion, tamIndividuo,pbb):
  CreaPoblacion = np.random.choice([0, 1], size=(numPoblacion, tamIndividuo), p=[1-pbb, pbb])
  fitness = np.sum(CreaPoblacion, axis=1)
  print(fitness)
  poblacion = np.column_stack((CreaPoblacion, fitness))
  return poblacion 
  print(poblacion)


def seleccion_ruleta(poblacion, fitness):
    totalFitness = np.sum(fitness)
    probabilidadSeleccion = fitness / totalFitness
    nuevap = np.random.choice(poblacion.shape[0], size=1, p=probabilidadSeleccion)
    return poblacion[nuevap]

def seleccion_torneo(poblacion, fitness):
  n = len(poblacion)
  seleccionados = []
  for i in range(3):
    seleccion = np.random.choice(n, 3, replace=False)
    ganador = seleccion[np.argmax(fitness[seleccion])]
  return np.array(poblacion[ganador])

def crucePunto(padre1, padre2, puntoCruce):
  hijo = np.concatenate((padre1[:puntoCruce], padre2[puntoCruce:]))
  hijo_fitness = np.sum(hijo)
  hijo = np.append(hijo, hijo_fitness)
  return hijo

def crucePuntoBoble(padre1, padre2, puntoCruce1, puntoCruce2):
    hijo = np.concatenate((padre1[:puntoCruce1], padre2[puntoCruce1:puntoCruce2], padre1[puntoCruce2:]))
    hijo_fitness = np.sum(hijo)
    hijo = np.append(hijo, hijo_fitness)
    return hijo

def cruceUniforme(individuo1, individuo2):
    mascara = np.random.choice([False, True], size=individuo1.shape)
    hijo = np.where(mascara, individuo1, individuo2)
    hijo_fitness = np.sum(hijo)
    hijo = np.append(hijo, hijo_fitness)
    return hijo

def mutacionFlipBit(individuo, pbb):
    for i in range(len(individuo)):
        if np.random.random() < pbb:
            individuo[i] = 1 - individuo[i]
    individuo_fitness = np.sum(individuo)
    individuo = np.append(individuo, individuo_fitness)
    return individuo

def mutacionIntercambio(individuo, pbb):
    genoma = individuo.copy()
    posiciones = np.random.choice(genoma.shape[0], size=2, replace=False)
    pos_1 = posiciones[0]
    pos_2 = posiciones[1]
    if np.random.uniform() < pbb:
        genoma[pos_1], genoma[pos_2] = genoma[pos_2], genoma[pos_1]
    genoma_fitness = np.sum(genoma)
    genoma = np.append(genoma, genoma_fitness)
    return genoma


def mutacion_scramble(individuo, pbb):
    nuevo_individuo = individuo.copy()
    if np.random.random() < pbb:
        punto_1 = np.random.randint(0, len(individuo))
        punto_2 = np.random.randint(0, len(individuo))
        while punto_2 == punto_1:
            punto_2 = np.random.randint(0, len(individuo))
        if punto_1 > punto_2:
            punto_1, punto_2 = punto_2, punto_1
        segmento = nuevo_individuo[punto_1:punto_2+1]
        np.random.shuffle(segmento)
        nuevo_individuo[punto_1:punto_2+1] = segmento
    nuevo_individuo_fitness = np.sum(nuevo_individuo)
    nuevo_individuo = np.append(nuevo_individuo, nuevo_individuo_fitness)
    return nuevo_individuo

def mutacion_inversion(individuo, pbb):
    if np.random.rand() < pbb:
        punto1 = np.random.randint(len(individuo))
        punto2 = np.random.randint(len(individuo))
        while punto1 == punto2:
            punto2 = np.random.randint(len(individuo))
        if punto1 > punto2:
            punto1, punto2 = punto2, punto1
        segmento = individuo[punto1:punto2+1]
        individuo[punto1:punto2+1] = segmento[::-1]
    individuo_fitness = np.sum(individuo)
    individuo = np.append(individuo, individuo_fitness)
    return individuo


# Tamaño de la población
tam_poblacion = 5

# Tamaño de cada individuo
tam_individuo = 10

# Probabilidad de que cada elemento sea 1
probabilidad = 0.5

# Generar matriz de población aleatoria con probabilidad de 1
poblacion = np.random.choice([0, 1], size=(tam_poblacion, tam_individuo), p=[1-probabilidad, probabilidad])

# Calcular la suma de cada individuo y agregarla como segunda columna de la matriz
suma_individuos = np.sum(poblacion, axis=1)
poblacion_con_suma = np.column_stack((poblacion, suma_individuos))


# Seleccionar nueva población por ruleta
fitness_poblacion = poblacion_con_suma[:, -1]
nueva_poblacion = seleccion_ruleta(poblacion_con_suma, fitness_poblacion)
nueva_poblacion1 = seleccion_ruleta(poblacion_con_suma, fitness_poblacion)

#print("Población original:\n", poblacion_con_suma)
print("Nueva población:\n", nueva_poblacion)
#print("Nueva población:\n", nueva_poblacion1)
print("hijo:\n",mutacion_inversion(nueva_poblacion[0][:-1],1))

Nueva población:
 [[0 0 1 1 1 0 0 1 1 1 6]]
hijo:
 [0 0 1 1 1 0 0 1 1 1 6]


In [ ]:
def seleccion_muestreo_universal_estocastico(poblacion):
    # Calcular el total de fitness
    total_fitness = np.sum(poblacion[:, -1])

    # Calcular el tamaño de la población
    tam_poblacion = poblacion.shape[0]

    # Calcular la separación entre los puntos de muestreo
    separacion = total_fitness / 2

    # Generar un número aleatorio entre 0 y la separación
    punto_muestreo = np.random.uniform(low=0, high=separacion)

    # Inicializar el índice de los individuos seleccionados
    idx_seleccionados = []

    # Inicializar el valor de fitness acumulado
    fitness_acumulado = 0

    # Realizar el muestreo universal estocástico
    for i in range(2):
        # Incrementar el valor de fitness acumulado
        fitness_acumulado += poblacion[i, -1]

        # Seleccionar al individuo correspondiente al punto de muestreo
        while punto_muestreo <= fitness_acumulado:
            idx_seleccionados.append(i)
            punto_muestreo += separacion

    # Devolver los índices de los individuos seleccionados
    ps = poblacion[idx_seleccionados]
    return ps

print(seleccion_muestreo_universal_estocastico(crear(10, 8, 0.2)))


[4 0 0 0 1 0 0 2 3 1]
[]
